- 有点乱，大家根据自己习惯瞎改改吧
- kernel替换完成后，不可逆，需要刷新重启
- 测试结果都是取得稳定状态下的迭代速度，大概在20步之后

In [2]:
import os
# os.environ['CUDA_VISIBLE_DEVICES'] = '7'
import torch
import sys
import importlib
# 如果测huggingface中没有模型，需要把路径导进来，不然import了kernel，使用AutoModel加载模型也没有用处
sys.path.append('/mnt/workspace/mdy/models')
module = importlib.import_module('glm-4-9b-chat.modeling_chatglm')
from transformers import AutoConfig, AutoModelForCausalLM
import time
from copy import deepcopy
from tqdm import tqdm

In [3]:

# config = AutoConfig.from_pretrained('/mnt/workspace/mdy/models/Qwen2.5-7B-Instruct')
dtype = torch.bfloat16
bs = 2
seq_len = 1536
iters = 50
model_path = '/mnt/workspace/mdy/models/glm-4-9b-chat'
# model_path = '/mnt/workspace/mdy/models/Qwen2.5-7B'
# model_path = '/mnt/workspace/mdy/models/Llama-3.2-3B-Instruct'
# model_path = '/mnt/workspace/mdy/models/Meta-Llama-3.1-8B-Instruct'

model = AutoModelForCausalLM.from_pretrained(model_path,
                                            _attn_implementation='flash_attention_2',
                                           device_map='cuda', torch_dtype=dtype, trust_remote_code=True)
# model = module.ChatGLMForConditionalGeneration.from_pretrained(model_path, _attn_implementation='flash_attention_2',
#                                            device_map='cuda', torch_dtype=dtype, trust_remote_code=True)
input_ids = torch.randint(10000, 20000, (bs, seq_len)).cuda()
pos_ids = torch.arange(seq_len)[None, :].expand(bs, -1).contiguous().cuda()
labels = input_ids
grad_list = []
for p in model.parameters():
    grad_list.append(p)
def zero_gard():
    for p in grad_list:
        if p.grad is not None:
            p.grad.zero_()


Loading checkpoint shards: 100%|██████████| 10/10 [00:09<00:00,  1.00it/s]


In [ ]:
torch.cuda.empty_cache()
t = time.time()
for i in tqdm(range(iters)):
    out = model(input_ids, position_ids=pos_ids, labels=labels)
    out.loss.backward()
t1 = time.time() - t
t1

  0%|          | 0/50 [00:00<?, ?it/s]

100%|██████████| 50/50 [00:47<00:00,  1.05it/s]


47.69300603866577

In [5]:
print(grad_list[10].grad[:5])
zero_gard()

tensor([-0.0024,  0.0112,  0.0121,  0.0072,  0.0044], device='cuda:0',
       dtype=torch.bfloat16)


In [1]:
from mdy_triton.replace_kernel import *

/mnt/workspace/mdy/miniforge/envs/mdy/lib/python3.10/site-packages/_distutils_hack/__init__.py:53: UserWarning: Reliance on distutils from stdlib is deprecated. Users must rely on setuptools to provide the distutils module. Avoid importing distutils or import setuptools first, and avoid setting SETUPTOOLS_USE_DISTUTILS=stdlib. Register concerns at https://github.com/pypa/setuptools/issues/new?template=distutils-deprecation.yml
  warnings.warn(
/mnt/workspace/mdy/miniforge/envs/mdy/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
torch.cuda.empty_cache()
t = time.time()
for i in tqdm(range(iters)):
    out = model(input_ids, position_ids=pos_ids,labels=labels)
    out.loss.backward()
t2 = time.time() - t
t2

 38%|███▊      | 19/50 [00:18<00:29,  1.07it/s]

In [9]:
print(grad_list[10].grad[:5])
zero_gard()

tensor([-0.0006,  0.0011,  0.0007,  0.0013,  0.0005], device='cuda:0',
       dtype=torch.bfloat16)
